In [1]:
!pip install yfinance timesfm==1.1.0


INFO: pip is looking at multiple versions of paxml to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax[cuda12] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax[cuda12] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is still looking at multiple versions of paxml to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install yfinance
!pip install utilsforecast


In [3]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['JAX_PMAP_USE_TENSORSTORE'] = 'false'

In [4]:
import timesfm
import gc
import numpy as np
import pandas as pd
from timesfm import patched_decoder
from timesfm import data_loader
import yfinance as yf

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [5]:
import numpy as np
import tensorflow as tf
import random
import os

# Set a seed value
seed_value = 42

# Set Python random seed
random.seed(seed_value)

# Set NumPy random seed
np.random.seed(seed_value)

# Set TensorFlow random seed
tf.random.set_seed(seed_value)

# For reproducibility in other areas, like OS operations
os.environ['PYTHONHASHSEED'] = str(seed_value)


In [6]:
from tqdm import tqdm
import dataclasses
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [7]:
# Loading TimesFM pretrained checkpoint
tfm = timesfm.TimesFm(
    context_len=64,
    horizon_len=1,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="gpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

(…)oint_1100000/descriptor/descriptor.pbtxt:   0%|          | 0.00/499 [00:00<?, ?B/s]

(…)nts/checkpoint_1100000/metadata/metadata:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

...

checkpoint:   0%|          | 0.00/814M [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 9.46 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 2.96 seconds.
Jitting decoding.
Jitted decoding in 32.87 seconds.


In [8]:
import pandas as pd
import yfinance as yf

# لیستی از نمادهای سهام معروف
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'V', 'NFLX']

# بارگذاری داده‌های بورسی روزانه برای هر سهام
all_data = []

for idx, ticker in enumerate(tickers, start=1):
    data = yf.download(ticker, start='1900-01-01', end='2024-01-01', interval='1d')
    if data.empty:
        print(f"Failed to download data for {ticker}")
        continue
    data.index = pd.to_datetime(data.index)
    data = data.dropna()
    data.reset_index(inplace=True)
    data.columns = data.columns.droplevel(1)
    data.rename(columns={'Date': 'date', 'Close': 'y'}, inplace=True)
    data['ticker'] = idx  # جایگزین کردن نام سهام با یک عدد یکتا

    # حذف 100 داده آخر برای اپل
    if ticker == 'AAPL':
        data = data.iloc[:-366]  # Remove the last 100 rows

    final_cols = ["date", "y", "ticker"]
    data = data[final_cols]
    all_data.append(data)
# ترکیب داده‌های تمام سهام‌ها در یک DataFrame
combined_data = pd.concat(all_data)

print(len(combined_data))
combined_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

64643


Price,date,y,ticker
0,1980-12-12 00:00:00+00:00,0.128348,1
1,1980-12-15 00:00:00+00:00,0.121652,1
2,1980-12-16 00:00:00+00:00,0.112723,1
3,1980-12-17 00:00:00+00:00,0.115513,1
4,1980-12-18 00:00:00+00:00,0.118862,1
...,...,...,...
5434,2023-12-22 00:00:00+00:00,486.760010,10
5435,2023-12-26 00:00:00+00:00,491.190002,10
5436,2023-12-27 00:00:00+00:00,491.790009,10
5437,2023-12-28 00:00:00+00:00,490.510010,10


In [9]:
data.columns

Index(['date', 'y', 'ticker'], dtype='object', name='Price')

In [10]:
apple_data = yf.download('AAPL', start='2000-01-01', end='2024-01-01', interval='1d')
apple_data.index = pd.to_datetime(apple_data.index)
apple_data = apple_data.dropna()
apple_data.reset_index(inplace=True)
apple_data.columns = apple_data.columns.droplevel(1)
apple_data.rename(columns={'Date': 'date', 'Close': 'y'}, inplace=True)
apple_data['ticker'] = 1  # استفاده از همان عدد یکتا برای اپل
apple_data = apple_data[["date", "y", "ticker"]].tail(366)
print(len(combined_data)) # انتخاب 100 داده آخر
print(len(apple_data))
# افزودن 100 داده آخر اپل به داده‌های ترکیبی
combined_data = pd.concat([combined_data, apple_data,apple_data])
print(len(combined_data))
# ذخیره‌سازی داده‌ها در یک فایل CSV
data_path = 'stocks_data.csv'
combined_data.to_csv(data_path, index=False)

print(f"Number of records after appending Apple data: {len(combined_data)}")

[*********************100%***********************]  1 of 1 completed


64643
366
65375
Number of records after appending Apple data: 65375


In [11]:
combined_data

Price,date,y,ticker
0,1980-12-12 00:00:00+00:00,0.128348,1
1,1980-12-15 00:00:00+00:00,0.121652,1
2,1980-12-16 00:00:00+00:00,0.112723,1
3,1980-12-17 00:00:00+00:00,0.115513,1
4,1980-12-18 00:00:00+00:00,0.118862,1
...,...,...,...
6032,2023-12-22 00:00:00+00:00,193.600006,1
6033,2023-12-26 00:00:00+00:00,193.050003,1
6034,2023-12-27 00:00:00+00:00,193.149994,1
6035,2023-12-28 00:00:00+00:00,193.580002,1


In [12]:
combined_data[-186:]

Price,date,y,ticker
5851,2023-04-05 00:00:00+00:00,163.759995,1
5852,2023-04-06 00:00:00+00:00,164.660004,1
5853,2023-04-10 00:00:00+00:00,162.029999,1
5854,2023-04-11 00:00:00+00:00,160.800003,1
5855,2023-04-12 00:00:00+00:00,160.100006,1
...,...,...,...
6032,2023-12-22 00:00:00+00:00,193.600006,1
6033,2023-12-26 00:00:00+00:00,193.050003,1
6034,2023-12-27 00:00:00+00:00,193.149994,1
6035,2023-12-28 00:00:00+00:00,193.580002,1


In [13]:
freq = 'D'
context_len = 64
pred_len = 1
batch_size = 32



In [14]:
train_boundary = len(combined_data )-366
val_boundary = len(combined_data )-186

In [15]:
ts_cols = [col for col in data.columns if col != "date"]
cat_cols=[col for col in ts_cols if col != "y"]

num_ts = 1
# Create TimeSeriesdata loader
dtl = data_loader.TimeSeriesdata(
      data_path=data_path,
      datetime_col="date",
      num_cov_cols=cat_cols,
      cat_cov_cols=None,
      ts_cols=["y"],
      train_range=[0, train_boundary],
      val_range=[train_boundary, val_boundary],
      test_range=[val_boundary, len(combined_data)],
      hist_len=context_len,
      pred_len=pred_len,
      batch_size=1,  # Change as per requirement
      freq=freq,
      normalize=True,
      epoch_len=None,
      holiday=False,
      permute=False,
  )
train_batches = dtl.tf_dataset(mode="train", shift=1).batch(batch_size)
val_batches = dtl.tf_dataset(mode="val", shift=pred_len)
test_batches = dtl.tf_dataset(mode="test", shift=pred_len)

In [16]:
for tbatch in tqdm(train_batches.as_numpy_iterator()):
    pass
print(tbatch[0].shape)
print(tbatch[1].shape)
print(tbatch[2].shape)

4059it [00:56, 71.35it/s]

(32, 1, 64)
(32, 8, 64)
(32, 1, 64)


In [17]:
from collections import defaultdict
# Download stock data from Yahoo Finance
ticker = 'AAPL'  # Example: Apple Inc.
data = yf.download(ticker, start="2023-01-01", end="2024-01-01")
data = data.dropna()

df = data[['Close']].reset_index()
df.columns = ['date', 'close']
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)


def get_batched_data_fn(batch_size: int = 32, context_len: int = 32, horizon_len: int = 1):
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(df) - (context_len + horizon_len), horizon_len):
        num_examples += 1
        context_end = start + context_len
        examples["inputs"].append(df["close"][start:context_end].tolist())
        examples["outputs"].append(df["close"][context_end:context_end + horizon_len].tolist())
        examples["dates"].append(df.index[start:context_end + horizon_len].tolist())  # Add dates to examples

    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {k: v[(i * batch_size): ((i + 1) * batch_size)] for k, v in examples.items()}

    return data_fn
context_len=64
input_patch_len=32
horizon_len=1
batch_size=1

input_data = get_batched_data_fn(batch_size=batch_size, context_len=context_len, horizon_len=horizon_len)

results = []
# Process in smaller sub-batches
for i, example in enumerate(input_data()):
    raw_forecast, _ = tfm.forecast(
        inputs=example["inputs"], freq=[0] * len(example["inputs"])
    )
    for j in range(len(example["inputs"])):
        result = {
            'input_index': j,
            'train_start_date': str(example["dates"][j][0]),
            'train_end_date': str(example["dates"][j][-2]),
            'predict_date': str(example["dates"][j][-1]),
            'raw_forecast': raw_forecast[j][0],
            'y_actual': example["outputs"][j][0],
            'batch_size': batch_size,
            'context_len': context_len,
            'input_patch_len': input_patch_len
        }
        results.append(result)
results_df = pd.DataFrame(results)
from sklearn.metrics import mean_squared_error, accuracy_score
mean_squared_error(results_df['y_actual'], results_df['raw_forecast'])


[*********************100%***********************]  1 of 1 completed


5.47923479300823

# MAE on the test split for the pretrained TimesFM model

In [18]:
mse_losses = []
for batch in tqdm(test_batches.as_numpy_iterator()):
    past = batch[0]
    actuals = batch[3]

    _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
    forecasts = forecasts[:, 0 : actuals.shape[1], 5]

    mse_losses.append(np.square(forecasts - actuals).mean())

print(f"MSE: {np.mean(mse_losses)}")


186it [00:03, 56.68it/s]


MSE: 0.0007428496610373259


# Fine-tuning the model

In [19]:
import jax
from jax import numpy as jnp
from praxis import pax_fiddle
from praxis import py_utils
from praxis import pytypes
from praxis import base_model
from praxis import optimizers
from praxis import schedules
from praxis import base_hyperparams
from praxis import base_layer
from paxml import tasks_lib
from paxml import trainer_lib
from paxml import checkpoints
from paxml import learners
from paxml import partitioning
from paxml import checkpoint_types

In [20]:
NestedMap = py_utils.NestedMap
WeightInit = base_layer.WeightInit
WeightHParams = base_layer.WeightHParams
InstantiableParams = py_utils.InstantiableParams
JTensor = pytypes.JTensor
NpTensor = pytypes.NpTensor
WeightedScalars = pytypes.WeightedScalars
instantiate = base_hyperparams.instantiate
LayerTpl = pax_fiddle.Config[base_layer.BaseLayer]
AuxLossStruct = base_layer.AuxLossStruct

AUX_LOSS = base_layer.AUX_LOSS
template_field = base_layer.template_field

# Standard prng key names
PARAMS = base_layer.PARAMS
RANDOM = base_layer.RANDOM

key = jax.random.PRNGKey(seed=1234)

In [21]:
model = pax_fiddle.Config(
    patched_decoder.PatchedDecoderFinetuneModel,
    name='patched_decoder_finetune',
    core_layer_tpl=tfm.model_p,
)

In [22]:

@pax_fiddle.auto_config
def build_learner() -> learners.Learner:
  return pax_fiddle.Config(
      learners.Learner,
      name='learner',
      loss_name='avg_qloss',
      optimizer=optimizers.Adam(
          epsilon=1e-7,
          clip_threshold=1e2,
          learning_rate=1e-2,
          lr_schedule=pax_fiddle.Config(
              schedules.Cosine,
              initial_value=1e-3,
              final_value=1e-4,
              total_steps=40000,
          ),
          ema_decay=0.9999,
      ),
      # Linear probing i.e we hold the transformer layers fixed.
      bprop_variable_exclusion=['.*/stacked_transformer_layer/.*'],
  )

In [23]:

task_p = tasks_lib.SingleTask(
    name='ts-learn',
    model=model,
    train=tasks_lib.SingleTask.Train(
        learner=build_learner(),
    ),
)

In [24]:
task_p.model.ici_mesh_shape = [1, 1, 1]
task_p.model.mesh_axis_names = ['replica', 'data', 'mdl']

DEVICES = np.array(jax.devices()).reshape([1, 1, 1])
MESH = jax.sharding.Mesh(DEVICES, ['replica', 'data', 'mdl'])

num_devices = jax.local_device_count()
print(f'num_devices: {num_devices}')
print(f'device kind: {jax.local_devices()[0].device_kind}')

num_devices: 1
device kind: Tesla T4


In [25]:
jax_task = task_p
key, init_key = jax.random.split(key)

# To correctly prepare a batch of data for model initialization (now that shape
# inference is merged), we take one devices*batch_size tensor tuple of data,
# slice out just one batch, then run the prepare_input_batch function over it.


def process_train_batch(batch):

    past_ts = batch[0].reshape(len(batch[2]) * num_ts, -1)
    actual_ts = batch[3].reshape(len(batch[2]) * num_ts, -1)
    return NestedMap(input_ts=past_ts, actual_ts=actual_ts)


def process_eval_batch(batch):
    past_ts = batch[0]
    actual_ts = batch[3]
    return NestedMap(input_ts=past_ts, actual_ts=actual_ts)


jax_model_states, _ = trainer_lib.initialize_model_state(
    jax_task,
    init_key,
    process_train_batch(tbatch),
    checkpoint_type=checkpoint_types.CheckpointType.GDA,
)

In [26]:
jax_model_states.mdl_vars['params']['core_layer'] = tfm._train_state.mdl_vars['params']
jax_vars = jax_model_states.mdl_vars
gc.collect()

0

In [27]:
jax_task = task_p


def train_step(states, prng_key, inputs):
  return trainer_lib.train_step_single_learner(
      jax_task, states, prng_key, inputs
  )


def eval_step(states, prng_key, inputs):
  states = states.to_eval_state()
  return trainer_lib.eval_step_single_learner(
      jax_task, states, prng_key, inputs
  )

key, train_key, eval_key = jax.random.split(key, 3)
train_prng_seed = jax.random.split(train_key, num=jax.local_device_count())
eval_prng_seed = jax.random.split(eval_key, num=jax.local_device_count())

p_train_step = jax.pmap(train_step, axis_name='batch')
p_eval_step = jax.pmap(eval_step, axis_name='batch')

In [28]:
replicated_jax_states = trainer_lib.replicate_model_state(jax_model_states)
replicated_jax_vars = replicated_jax_states.mdl_vars

In [32]:
best_eval_loss = 1e7
step_count = 0
patience = 0
NUM_EPOCHS = 100
PATIENCE = 5
TRAIN_STEPS_PER_EVAL = 1000
CHECKPOINT_DIR='/content'

In [ ]:
def reshape_batch_for_pmap(batch, num_devices):
  def _reshape(input_tensor):
    bsize = input_tensor.shape[0]
    residual_shape = list(input_tensor.shape[1:])
    nbsize = bsize // num_devices
    return jnp.reshape(input_tensor, [num_devices, nbsize] + residual_shape)

  return jax.tree.map(_reshape, batch)

In [ ]:
!rm -rf /content/checkpoint*


In [ ]:
import pickle
save_count = 0
for epoch in range(NUM_EPOCHS):
    print(f"__________________Epoch: {epoch}__________________", flush=True)

    # Train phase
    train_its = train_batches.as_numpy_iterator()
    if patience >= PATIENCE:
        print("Early stopping.", flush=True)
        break

    train_losses = []  # Initialize train losses for the epoch
    for batch in train_its:
        if patience >= PATIENCE:
            print("Early stopping.", flush=True)
            break

        tbatch = process_train_batch(batch)
        tbatch = reshape_batch_for_pmap(tbatch, num_devices)
        replicated_jax_states, step_fun_out = p_train_step(
            replicated_jax_states, train_prng_seed, tbatch
        )
        train_losses.append(step_fun_out.loss[0])

        # Increment step count
        step_count += 1

    # Calculate and print average train loss for the epoch
    avg_train_loss = np.mean(train_losses)
    print(f"Train loss at end of epoch {epoch}: {avg_train_loss}", flush=True)

    # Eval phase at the end of the epoch
    print("Starting eval.", flush=True)
    val_its = val_batches.as_numpy_iterator()
    eval_losses = []  # Initialize eval losses for the epoch

    for ev_batch in tqdm(val_its):
        ebatch = process_eval_batch(ev_batch)
        ebatch = reshape_batch_for_pmap(ebatch, num_devices)
        _, step_fun_out = p_eval_step(replicated_jax_states, eval_prng_seed, ebatch)
        eval_losses.append(step_fun_out.loss[0])

    # Calculate and print average eval loss for the epoch
    mean_loss = np.mean(eval_losses)
    print(f"Eval loss at end of epoch {epoch}: {mean_loss}", flush=True)

    # Check for checkpoint saving and patience logic
    if mean_loss < best_eval_loss or np.isnan(mean_loss):
        best_eval_loss = mean_loss
        print("Saving checkpoint.")
        jax_state_for_saving = py_utils.maybe_unreplicate_for_fully_replicated(
            replicated_jax_states
        )
        save_count += 1
        if save_count % 5 == 0:
            print("Unwaited save to Google Drive.")
            checkpoints.save_checkpoint(
                jax_state_for_saving, '/content/drive/MyDrive', overwrite=True,
            )
            with open(os.path.join( '/content/drive/MyDrive', 'train_state_info.pkl'), 'wb') as f:
               pickle.dump({'step_count': step_count, 'epoch': epoch}, f)
               print(f"Checkpoint saved at step {step_count}, epoch {epoch}")
        checkpoints.save_checkpoint(
            jax_state_for_saving, CHECKPOINT_DIR, overwrite=True
        )
        patience = 0  # Reset patience since we improved
        del jax_state_for_saving
        gc.collect()
    else:
        patience += 1
        print(f"patience: {patience}")

    # Early stopping check
    if patience >= PATIENCE:
        print("Early stopping.", flush=True)
        break


__________________Epoch: 0__________________
Train loss at end of epoch 0: 0.08757846057415009
Starting eval.


180it [00:14, 12.18it/s]

Eval loss at end of epoch 0: 0.23761041462421417



        shapes of train_state  as the unpadded shapes.


Saving checkpoint.


...

Train loss at end of epoch 56: 0.06523045897483826
Starting eval.


180it [00:01, 103.39it/s]

Eval loss at end of epoch 56: 0.21437481045722961



        shapes of train_state  as the unpadded shapes.


Saving checkpoint.


In [30]:
!gdown --folder "https://drive.google.com/drive/folders/11Imt8u3vu_BQoDJc183FLFhC5_t_EIcP?usp=drive_link"

Retrieving folder contents
Retrieving folder 11M0ilAo8UrKkUV_4fOT_v8vW3A3FDxe8 metadata
Processing file 11PXhnTv12PoHKA4_gIptdJizRjP5VzQT metadata
Retrieving folder 11IzYCU6CSmsMgH-gmnct_fKkA3rb8r7a state
Processing file 11K7Wp0n9szIPUY6JLoPkN1BwL-RhvURh checkpoint
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=11PXhnTv12PoHKA4_gIptdJizRjP5VzQT
To: /content/checkpoint_211068/metadata/metadata
100% 43.9k/43.9k [00:00<00:00, 96.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=11K7Wp0n9szIPUY6JLoPkN1BwL-RhvURh
From (redirected): https://drive.google.com/uc?id=11K7Wp0n9szIPUY6JLoPkN1BwL-RhvURh&confirm=t&uuid=1dadf90a-97c6-49c8-90a8-2f3209239957
To: /content/checkpoint_211068/state/checkpoint
100% 895M/895M [00:19<00:00, 45.6MB/s]
Download completed


In [33]:
train_state = checkpoints.restore_checkpoint(jax_model_states, CHECKPOINT_DIR)
print(train_state.step)
tfm._train_state.mdl_vars['params'] = train_state.mdl_vars['params']['core_layer']
tfm.jit_decode()

ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


211068
Jitting decoding.
Jitted decoding in 21.65 seconds.


In [34]:
mse_losses = []
for batch in tqdm(test_batches.as_numpy_iterator()):
    past = batch[0]
    actuals = batch[3]
    _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
    forecasts = forecasts[:, 0 : actuals.shape[1], 5]
    mse_losses.append(np.square(forecasts - actuals))

print(f"Mse: {np.mean(mse_losses)}")

186it [00:03, 53.54it/s]


Mse: 0.0005340901552699506


In [35]:
from collections import defaultdict
# Download stock data from Yahoo Finance
ticker = 'AAPL'  # Example: Apple Inc.
data = yf.download(ticker, start="2023-01-01", end="2024-01-01")
data = data.dropna()

df = data[['Close']].reset_index()
df.columns = ['date', 'close']
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)


def get_batched_data_fn(batch_size: int = 32, context_len: int = 32, horizon_len: int = 1):
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(df) - (context_len + horizon_len), horizon_len):
        num_examples += 1
        context_end = start + context_len
        examples["inputs"].append(df["close"][start:context_end].tolist())
        examples["outputs"].append(df["close"][context_end:context_end + horizon_len].tolist())
        examples["dates"].append(df.index[start:context_end + horizon_len].tolist())  # Add dates to examples

    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {k: v[(i * batch_size): ((i + 1) * batch_size)] for k, v in examples.items()}

    return data_fn
context_len=64
input_patch_len=32
horizon_len=1
batch_size=1

input_data = get_batched_data_fn(batch_size=batch_size, context_len=context_len, horizon_len=horizon_len)

results = []
# Process in smaller sub-batches
for i, example in enumerate(input_data()):
    raw_forecast, _ = tfm.forecast(
        inputs=example["inputs"], freq=[0] * len(example["inputs"])
    )
    for j in range(len(example["inputs"])):
        result = {
            'input_index': j,
            'train_start_date': str(example["dates"][j][0]),
            'train_end_date': str(example["dates"][j][-2]),
            'predict_date': str(example["dates"][j][-1]),
            'raw_forecast': raw_forecast[j][0],
            'y_actual': example["outputs"][j][0],
            'batch_size': batch_size,
            'context_len': context_len,
            'input_patch_len': input_patch_len
        }
        results.append(result)
results_df = pd.DataFrame(results)
from sklearn.metrics import mean_squared_error, accuracy_score
mean_squared_error(results_df['y_actual'], results_df['raw_forecast'])


[*********************100%***********************]  1 of 1 completed


3.9618054968606984